<a href="https://colab.research.google.com/github/rzendonadi/project/blob/main/tobler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing

In [ ]:
!pip install geopandas
!pip install cartoframes
!pip install h3
!pip install libpysal
!pip install tobler
!pip install splot
!pip install git+https://github.com/pysal/region.git#egg=region
!apt install libspatialindex-dev
!pip install rtree
!pip install geovoronoi


import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tobler
import geovoronoi

from cartoframes import read_carto, to_carto
from cartoframes.auth import set_default_credentials
from cartoframes.viz import *
from h3 import h3
from libpysal.weights import W, Rook
from scipy.stats import percentileofscore
from shapely import wkt
from shapely.geometry import mapping, Polygon
from shapely.ops import cascaded_union
from sklearn.cluster import KMeans
from tobler.area_weighted import area_interpolate
from splot.libpysal import plot_spatial_weights
from region.max_p_regions.heuristics import MaxPRegionsHeu

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


# Geopandas

In [ ]:
#import local files
#from google.colab import files
#files.upload()

In [ ]:
#import drive files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#searching file on the drive
!ls "/content/drive/My Drive/Colab Notebooks/data"

hexagon.csv  polygon.csv  tobler.csv


In [ ]:
hex = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/hexagon.csv")
hex['geometry'] = list(map(wkt.loads, hex['geometry']))
hex = gpd.GeoDataFrame(hex, crs='epsg:4326')
hex.head()

,hex_id,type,cd_geocodi,cd_geocodd,district_name,cd_geocodm,city_name,uf,ddd,class,geometry
0,8aa8100e0c87fff,URBANO,355030877000009,355030877,SAÚDE,3550308,SÃO PAULO,SP,11,B1,"POLYGON ((-46.64287 -23.61772, -46.64228 -23.6..."
1,8aa8100e0507fff,URBANO,355030877000003,355030877,SAÚDE,3550308,SÃO PAULO,SP,11,B1,"POLYGON ((-46.64278 -23.61465, -46.64219 -23.6..."
2,8aa8100e051ffff,URBANO,355030877000003,355030877,SAÚDE,3550308,SÃO PAULO,SP,11,B1,"POLYGON ((-46.64231 -23.61358, -46.64172 -23.6..."
3,8aa8100e050ffff,URBANO,355030877000003,355030877,SAÚDE,3550308,SÃO PAULO,SP,11,B1,"POLYGON ((-46.64363 -23.61373, -46.64304 -23.6..."
4,8aa8100e46cffff,URBANO,355030877000095,355030877,SAÚDE,3550308,SÃO PAULO,SP,11,B1,"POLYGON ((-46.65002 -23.62264, -46.64942 -23.6..."


In [ ]:
pol = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/polygon.csv")
pol['geometry'] = list(map(wkt.loads, pol['geometry']))
pol = gpd.GeoDataFrame(pol, crs='epsg:4326')
pol.head()

,type,cd_geocodi,cd_geocodd,district_name,cd_geocodm,city_name,class,classe_int,uf,ddd,population,male_population,female_population,geometry
0,URBANO,355030877000009,355030877,SAÚDE,3550308,SÃO PAULO,B1,2,SP,11,697,310,387,"POLYGON ((-46.64410 -23.61805, -46.64416 -23.6..."
1,URBANO,355030877000003,355030877,SAÚDE,3550308,SÃO PAULO,B1,2,SP,11,374,177,197,"POLYGON ((-46.64421 -23.61442, -46.64426 -23.6..."
2,URBANO,355030877000095,355030877,SAÚDE,3550308,SÃO PAULO,B1,2,SP,11,294,140,154,"POLYGON ((-46.65174 -23.62368, -46.65179 -23.6..."
3,URBANO,355030877000065,355030877,SAÚDE,3550308,SÃO PAULO,B1,2,SP,11,213,93,120,"POLYGON ((-46.64790 -23.62177, -46.64795 -23.6..."
4,URBANO,355030877000142,355030877,SAÚDE,3550308,SÃO PAULO,B2,3,SP,11,116,52,64,"POLYGON ((-46.64185 -23.62889, -46.64190 -23.6..."


In [ ]:
result_area = tobler.area_weighted.area_interpolate(pol, hex, intensive_variables=['population'])

/usr/local/lib/python3.6/dist-packages/tobler/area_weighted/area_interpolate.py:253: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  den = source_df["geometry"].area.values


In [ ]:
display(result_area)

,population,geometry
0,263.879465,"POLYGON ((-46.64287 -23.61772, -46.64228 -23.6..."
1,145.689734,"POLYGON ((-46.64278 -23.61465, -46.64219 -23.6..."
2,181.954729,"POLYGON ((-46.64231 -23.61358, -46.64172 -23.6..."
3,149.197961,"POLYGON ((-46.64363 -23.61373, -46.64304 -23.6..."
4,118.018506,"POLYGON ((-46.65002 -23.62264, -46.64942 -23.6..."
...,...,...
604,182.336588,"POLYGON ((-46.63696 -23.60988, -46.63636 -23.6..."
605,58.280992,"POLYGON ((-46.64976 -23.59920, -46.64917 -23.5..."
606,69.745028,"POLYGON ((-46.64891 -23.60012, -46.64832 -23.5..."
607,25.632677,"POLYGON ((-46.64844 -23.59904, -46.64785 -23.5..."


In [ ]:
result_area['centroid'] = result_area.geometry.centroid
enriched_grid = gpd.sjoin(hex, result_area.set_geometry('centroid').drop(columns='geometry')).drop(columns='index_right')

In [ ]:
display(enriched_grid)

In [ ]:
enriched_grid.to_csv('tobler.csv')
!cp tobler.csv "/content/drive/My Drive/Colab Notebooks/data/tobler.csv"